<a href="https://colab.research.google.com/github/armandordorica/MIE1516_A1_Variable_Elimination/blob/master/MIE1516_A1_Q3_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [305]:
!pip install PrettyTable
!pip install truths

  Created wheel for truths: filename=truths-1.2-cp36-none-any.whl size=7876 sha256=dc421fc839e2ec8d23d22606d9e0e6b5bc921541887e9f70e6202262e2145beb
  Stored in directory: /root/.cache/pip/wheels/7b/86/9b/a9954e65607f963fc53e03f67d81950baabdd620685dd1127d
Successfully built truths


In [0]:
import prettytable
import numpy as np
import pandas as pd
pretty=prettytable.PrettyTable()

In [0]:
def toggle(var):
  if var == 0:
    return 1
  if var == 1:
    return 0

def boolean_truth_table(num_vars):
  truth_table = []
  num_vars = num_vars
  num_cols = num_vars
  var = 1
  num_rows = 2**num_vars
  for col_num in range(0,num_cols):
    truth_table.append([])
    col_num = col_num+1
    # print("col: {} toggle every {} values".format(col_num, 2**(num_vars-col_num)))
    for row_num in range(0, num_rows):
      if row_num%2**(num_vars-col_num)==0:
        var = toggle(var)
      truth_table[col_num-1].append(var)
      # print(truth_table[col_num-1])
      # print(var)
  return truth_table

def generate_truth_table(list_of_variables):
  d = dict()
  num_vars = len(list_of_variables)
  for i in range(0, num_vars): 
    d[str(list_of_variables[i])] = boolean_truth_table(len(list_of_variables))[i]
  df = pd.DataFrame(data=d)
  return df


In [0]:
class MyBayesianModel: 
  def __init__(self, list_of_edges):
    self.list_of_edges = list(list_of_edges)
    self.tabular_cpds=[]

  def get_variables(self):
      list_of_tuples = list(self.list_of_edges)
      list_of_items = [item for t in list_of_tuples for item in t] 
      list_set = set(list_of_items) 
      # convert the set to the list 
      unique_list_of_vars = (list(list_set))
      print(unique_list_of_vars)

  def available_cpds(self):
    for i in range(0, len(self.tabular_cpds)):
      self.tabular_cpds[i].print_factor()

  def eliminate_variable(self, variable_to_eliminate):
    print("Marginalize on {}".format(variable_to_eliminate))
    print("Multiply factors:")
    for i in range(0,len(self.tabular_cpds)):
      if self.tabular_cpds[i].get_factor().contains_var(variable_to_eliminate):
        self.tabular_cpds[i].print_factor()

    print("CPDs are:\n")
    for i in range(0,len(self.tabular_cpds)):
      if self.tabular_cpds[i].get_factor().contains_var(variable_to_eliminate):
        self.tabular_cpds[i].print_cpd()

In [0]:
class Factor:
  def __init__(self, indep_var, dep_vars=[]):
    self.indep_var = indep_var
    self.dep_vars = dep_vars
  
  def print_factor(self):
    if len(self.dep_vars)>0:
      self.dep_vars = set(self.dep_vars)
      self.dep_vars = list(self.dep_vars)
      self.dep_vars.sort()
      #print(self.dep_vars)
      dep_vars = str(self.dep_vars[0])
      for i in range (1, len(self.dep_vars)):
        dep_vars = dep_vars + "," + self.dep_vars[i]
      print("P({}|{})".format(self.indep_var, dep_vars))
    if len(self.dep_vars)==0:
      print("P({})".format(self.indep_var))

  def vars_in_factor(self):
    factors = list()
    factors.append(str(self.indep_var))
    for i in range (0, len(self.dep_vars)):
      factors.append(self.dep_vars[i])
    
    list_set = set(factors) 
    # convert the set to the list 
    unique_list_of_vars = list(list_set)
    unique_list_of_vars.sort(reverse=False)
    # print(unique_list_of_vars)
    return unique_list_of_vars

  def contains_var(self, variable):
    if variable in self.vars_in_factor():
      return True
    else:
      return False

  

In [0]:
class TabularCPD: 
  def __init__(self, variable, variable_card, values, evidence='', evidence_card=''): 
    self.variable = variable 
    self.variable_card = variable_card
    self.values = values
    self.evidence = evidence
    self.evidence_card = evidence_card

  # Initializing factors of the CPD depending on the format (whether evidence is provided or not)
    if len(self.evidence)>0:
      self.factors = []
      self.factors.append(Factor(self.variable, self.evidence))
    
    if len(self.evidence)==0:
      self.factors = []
      self.factors.append(Factor(self.variable))
    
    model.tabular_cpds.append(self)
  def print_cpd(self):
    if len(self.evidence) > 0:
      print("P({}|{})".format(self.variable, self.evidence[0]))
      pretty=prettytable.PrettyTable()
      pretty.add_column("{}".format(self.variable), ["{}(0)".format(self.variable),"{}(1)".format(self.variable)], align='l',valign='t')
      
      self.values = np.array(self.values)
      self.values = self.values.transpose()

      for i in range(0,len(self.values)):
        pretty.add_column("{}({})".format(self.evidence[0], i), self.values[i], align='l',valign='t')   
      print(pretty)

    else: 
      print("P({})".format(self.variable))
      pretty=prettytable.PrettyTable()
      pretty.add_column("{}".format(self.variable), ["{}(0)".format(self.variable),"{}(1)".format(self.variable)], align='l',valign='t')
      pretty.add_column("Pr", [item for t in self.values for item in t], align='l',valign='t')
      print(pretty)

  def get_factor(self):
    return self.factors[0]

  def print_factor(self):
    self.factors[0].print_factor()
  
  def to_pandas_df(self): 
    factor = self.get_factor()
    list_of_vars = []
    list_of_vars.append(self.get_factor().indep_var)
    if len(self.get_factor().dep_vars) > 0: 
      for i in range(0, len(self.get_factor().dep_vars)): 
        list_of_vars.append(self.get_factor().dep_vars[i])
      df = generate_truth_table(list_of_vars)
      df['Pr']= None

      example_array = self.values

      for a in range(0,2): 
        for b in range(0,2):
          row_index = df[(df[str(list_of_vars[0])]==a) & (df[str(list_of_vars[1])]==b)].index[0]
          df.set_value(row_index, 'Pr', example_array[a][b])
    if len(self.get_factor().dep_vars) == 0:
      example_array = self.values
      factor = self.get_factor()
      list_of_vars = []
      list_of_vars.append(self.get_factor().indep_var)
      df = generate_truth_table(list_of_vars)
      df['Pr']= None
      for i in range(0,self.variable_card):
        row_index = df[df[str(list_of_vars[0])]==i].index[0]
        # print(i, row_index)
        df.set_value(row_index, 'Pr', example_array[0][i])
      # print(list_of_vars)
    return df

In [0]:
  def eliminate_variable(list_of_factors, variable_to_eliminate):
    print("Marginalize on {}".format(variable_to_eliminate))
    print("Multiply factors:")
    for i in range(0,len(list_of_factors)):
      if list_of_factors[i].get_factor().contains_var(variable_to_eliminate):
        list_of_factors[i].print_factor()

    print("CPDs are:\n")
    for i in range(0,len(list_of_factors)):
      if list_of_factors[i].get_factor().contains_var(variable_to_eliminate):
        list_of_factors[i].print_cpd()

In [0]:
model = MyBayesianModel([('C', 'B'), ('B', 'A')])
cpd_c = TabularCPD(variable='C', variable_card = 2, values=[[0.8,0.2]])
cpd_a = TabularCPD(variable='A', variable_card=2, 
                   values=[[0.3, 0.9], 
                           [0.7, 0.1]],
                   evidence = ['B'], 
                   evidence_card=[2] 
                   )
cpd_b = TabularCPD(variable='B', variable_card=2, 
                   values=[[0.5, 0.4], 
                           [0.5, 0.6]],
                   evidence = ['C'], 
                   evidence_card=[2] 
                   )

In [682]:
cpd_b.to_pandas_df()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:60: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead


,B,C,Pr
0,0,0,0.5
1,0,1,0.4
2,1,0,0.5
3,1,1,0.6


In [683]:
cpd_a.to_pandas_df()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:60: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead


,A,B,Pr
0,0,0,0.3
1,0,1,0.9
2,1,0,0.7
3,1,1,0.1


In [684]:
cpd_c.to_pandas_df()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:71: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead


,C,Pr
0,0,0.8
1,1,0.2
